In [1]:
import csv
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', save_best_only=True)  # validation의 정확도 최대일때 저장
es = EarlyStopping(monitor='val_accuracy',mode='max',patience=5) 
# tvalidation의 정확도 최대일때. patience는 기준되는 값이 연속으로 몇 번 이상 향상되지 않을 때 종료시킬 것인지

In [2]:
df = pd.read_csv('C:/git_project/natural/data/mecab/X_train.csv')
df

,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,44
0,0,0,0,0,0,0,0,0,0,0,...,0,0,35,76,2528,45,245,22,39,717
1,0,0,0,0,0,0,0,0,0,0,...,698,4,122,1482,53,844,907,17,49,384
2,0,0,0,0,0,0,0,0,0,0,...,0,25,206,573,14,2,37,12,264,308
3,0,0,0,0,0,0,0,0,0,0,...,7131,161,4063,309,91,2,20,3,68,3391
4,0,0,0,0,0,0,0,0,0,0,...,61,10835,12697,7,421,9,1,4064,1613,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155934,12,4739,1120,19,15,1741,4,132,4,14,...,192,94,158,50,537,80,16,296,508,171
155935,0,0,0,0,0,0,0,0,0,0,...,4176,681,18,6,3,110,7,17220,49,3
155936,0,0,0,0,0,0,0,0,0,0,...,118,14,5,79,10,421,9,898,13,690
155937,0,0,0,0,0,0,0,0,0,0,...,9,1959,2,22,8,1407,421,9,105,171


In [3]:
X_train_data = pd.read_csv('C:/git_project/natural/data/mecab/X_train.csv')   # 데이터 불러오기
X_test_data = pd.read_csv('C:/git_project/natural/data/mecab/X_test.csv')
y_train_data = pd.read_csv('C:/git_project/natural/data/mecab/y_train.csv')   # 데이터 불러오기
y_test_data = pd.read_csv('C:/git_project/natural/data/mecab/y_test.csv')

In [4]:
X_train_data.dropna(how='any', inplace=True)  # null값 탈락
X_test_data.dropna(how='any', inplace=True)
y_train_data.dropna(how='any', inplace=True)
y_test_data.dropna(how='any', inplace=True)

In [5]:
train_X = np.array(X_train_data)       # array로 변경하여 tensor
test_X = np.array(X_test_data)
train_X, test_X

(array([[  0,   0,   0, ...,  22,  39, 717],
        [  0,   0,   0, ...,  17,  49, 384],
        [  0,   0,   0, ...,  12, 264, 308],
        ...,
        [  0,   0,   0, ..., 898,  13, 690],
        [  0,   0,   0, ...,   9, 105, 171],
        [  0,   0,   0, ..., 118, 608, 828]], dtype=int64),
 array([[   0,    0,    0, ...,    0,  825,  152],
        [   0,    0,    0, ..., 3358,   77,  930],
        [   0,    0,    0, ...,   32,   11,    2],
        ...,
        [   0,    0,    0, ...,    6,   15, 2260],
        [   0,    0,    0, ...,   66,  268,   46],
        [   0,    0,    0, ...,  177,   74, 1010]], dtype=int64))

In [6]:
train_y = np.array(y_train_data)
test_y = np.array(y_test_data)
train_y, test_y

(array([[0],
        [1],
        [0],
        ...,
        [1],
        [1],
        [1]], dtype=int64),
 array([[1],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]], dtype=int64))

# Model Building softsign

In [21]:
vocab_size = 21854                                                   #전처리 이후 사이즈 확인

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 32, mask_zero=True))   # 임베딩 32
model.add(keras.layers.Dropout(0.5))                                 # dropout 0.2 - 0.5사이
model.add(keras.layers.Bidirectional(keras.layers.GRU(64)))          # 노드  64
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(16, activation='softsign'))             # softsign, tanh로 돌려봄
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 32)          699328    
                                                                 
 dropout_12 (Dropout)        (None, None, 32)          0         
                                                                 
 bidirectional_4 (Bidirectio  (None, 128)              37632     
 nal)                                                            
                                                                 
 dropout_13 (Dropout)        (None, 128)               0         
                                                                 
 dense_8 (Dense)             (None, 16)                2064      
                                                                 
 dropout_14 (Dropout)        (None, 16)                0         
                                                      

In [22]:
model.compile(optimizer=keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [23]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_data, y_train_data))
val_dataset = tf.data.Dataset.from_tensor_slices((X_test_data, y_test_data))

train_dataset = train_dataset.shuffle(buffer_size=1024, reshuffle_each_iteration=True).batch(128)  # batch_size 64,128,256 확인
val_dataset = val_dataset.batch(1000)

train_dataset = train_dataset.prefetch(4)

In [25]:
history = model.fit(train_dataset, epochs=10, validation_data=val_dataset, verbose=1,callbacks=[mc])
#callbacks로 체크포인트 확인, epochs 10,20회 반복으로 시도

Epoch 1/10
1219/1219 [==============================] - 99s 81ms/step - loss: 0.2370 - accuracy: 0.9105 - val_loss: 0.3603 - val_accuracy: 0.8407
Epoch 2/10
1219/1219 [==============================] - 108s 89ms/step - loss: 0.2294 - accuracy: 0.9133 - val_loss: 0.3814 - val_accuracy: 0.8421
Epoch 3/10
1219/1219 [==============================] - 109s 89ms/step - loss: 0.2235 - accuracy: 0.9162 - val_loss: 0.3700 - val_accuracy: 0.8446
Epoch 4/10
1219/1219 [==============================] - 116s 95ms/step - loss: 0.2162 - accuracy: 0.9185 - val_loss: 0.3643 - val_accuracy: 0.8463
Epoch 5/10
1219/1219 [==============================] - 110s 90ms/step - loss: 0.2097 - accuracy: 0.9214 - val_loss: 0.3791 - val_accuracy: 0.8461
Epoch 6/10
1219/1219 [==============================] - 112s 92ms/step - loss: 0.2055 - accuracy: 0.9226 - val_loss: 0.3782 - val_accuracy: 0.8447
Epoch 7/10
1219/1219 [==============================] - 114s 94ms/step - loss: 0.2015 - accuracy: 0.9251 - val_loss: 0.

In [38]:
from keras.models import load_model
model.save('GRU_mecab_softsign_model.h5')

In [39]:
loaded_model = load_model('GRU_mecab_softsign_model.h5')
print("\n softsign 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test_data, y_test_data)[1]))

1527/1527 [==============================] - 14s 8ms/step - loss: 0.3580 - accuracy: 0.8430

 softsign 테스트 정확도: 0.8430


# Model Building - tanh

In [32]:
vocab_size = 21854                                                   #전처리 이후 사이즈 확인

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 32, mask_zero=True))   # 임베딩 32
model.add(keras.layers.Dropout(0.5))                                 # dropout 0.2 - 0.5사이
model.add(keras.layers.Bidirectional(keras.layers.GRU(64)))          # 노드  64
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(16, activation='tanh'))             # softsign, tanh로 돌려봄
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, None, 32)          699328    
                                                                 
 dropout_20 (Dropout)        (None, None, 32)          0         
                                                                 
 bidirectional_7 (Bidirectio  (None, 128)              37632     
 nal)                                                            
                                                                 
 dropout_21 (Dropout)        (None, 128)               0         
                                                                 
 dense_13 (Dense)            (None, 16)                2064      
                                                                 
 dropout_22 (Dropout)        (None, 16)                0         
                                                      

In [33]:
model.compile(optimizer=keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [34]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_data, y_train_data))
val_dataset = tf.data.Dataset.from_tensor_slices((X_test_data, y_test_data))

train_dataset = train_dataset.shuffle(buffer_size=1024, reshuffle_each_iteration=True).batch(128)  # batch_size 64,128,256 확인
val_dataset = val_dataset.batch(1000)

train_dataset = train_dataset.prefetch(4)

In [35]:
history = model.fit(train_dataset, epochs=10, validation_data=val_dataset, verbose=1,callbacks=[mc])
#callbacks로 체크포인트 확인, epochs 10,20회 반복으로 시도

Epoch 1/10
1219/1219 [==============================] - 431s 321ms/step - loss: 0.4220 - accuracy: 0.8090 - val_loss: 0.5141 - val_accuracy: 0.7354
Epoch 2/10
1219/1219 [==============================] - 266s 218ms/step - loss: 0.3530 - accuracy: 0.8534 - val_loss: 0.4556 - val_accuracy: 0.7738
Epoch 3/10
1219/1219 [==============================] - 154s 126ms/step - loss: 0.3305 - accuracy: 0.8660 - val_loss: 0.4240 - val_accuracy: 0.8048
Epoch 4/10
1219/1219 [==============================] - 155s 127ms/step - loss: 0.3095 - accuracy: 0.8761 - val_loss: 0.4501 - val_accuracy: 0.8115
Epoch 5/10
1219/1219 [==============================] - 155s 127ms/step - loss: 0.2940 - accuracy: 0.8843 - val_loss: 0.3781 - val_accuracy: 0.8260
Epoch 6/10
1219/1219 [==============================] - 153s 125ms/step - loss: 0.2773 - accuracy: 0.8914 - val_loss: 0.3761 - val_accuracy: 0.8284
Epoch 7/10
1219/1219 [==============================] - 154s 126ms/step - loss: 0.2674 - accuracy: 0.8950 - val_

In [40]:
from keras.models import load_model
model.save('GRU_mecab_tanh_model.h5')

In [37]:
loaded_model = load_model('GRU_mecab_tanh_model.h5')
print("\n tanh 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test_data, y_test_data)[1]))

1527/1527 [==============================] - 18s 10ms/step - loss: 0.3580 - accuracy: 0.8430

 테스트 정확도: 0.8430


### 하이퍼 파라미터 조절
#### OKT, tanh 사용 epochs 10회씩
drop out ---> 0.2 
batch size --->   256 =>128
layer ---->  32 , 중간 GRU(64)는 건들지 않음,  16


drop out --->    0.4
batch size ---> 128
layer ---->  32  , 중간 GRU(64)는 건들지 않음,  16


drop out --->  0.4 
batch size --->  128
layer ---->  32 =>16 , 중간 GRU(64)는 건들지 않음,  16=>8


drop out --->   0.5
batch size --->   128
layer ---->  16 , 중간 GRU(64)는 건들지 않음,  8


drop out --->   0.5
batch size --->   128
layer ---->  8 , 중간 GRU(64)는 건들지 않음,  4


drop out --->   0.5
batch size --->   64
layer ---->  8 , 중간 GRU(64)는 건들지 않음,  4


drop out --->   0.5
batch size --->   128
layer ---->  32 , 중간 GRU(64)는 건들지 않음,  16    =======> 최종 하이퍼파라미터

### 이후 
최종 하이퍼 파라미터 사용

mecab

softsign과 tanh 비교